Extract images

In [35]:
import cv2
import os
 
 
def video_to_hsv_frames(video_path, output_folder):
    cap = cv2.VideoCapture(video_path)
 
    frame_count = 0
 
    while True:
        ret, frame = cap.read()
 
        if not ret:
            break
 
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
 
        hsv_frame_path = os.path.join(output_folder, f'frame_{frame_count:04d}.png')
        cv2.imwrite(hsv_frame_path, hsv_frame)
 
        frame_count += 1
 
    cap.release()
    print(f'Extracted {frame_count} frames and converted them to HSV.')
 
 
video_path = r"C:\Users\Radcoflex-Purchase\Desktop\COLLEGE\SEM 7\Image and Video Analytics\lab 4/coke.mp4"  
output_folder = r"C:\Users\Radcoflex-Purchase\Desktop\COLLEGE\SEM 7\Image and Video Analytics\lab 4/cola"  
 
video_to_hsv_frames(video_path, output_folder)

Extracted 403 frames and converted them to HSV.


Segment

In [10]:
input_folder = r"C:\Users\Radcoflex-Purchase\Desktop\COLLEGE\SEM 7\Image and Video Analytics\lab 4\cola"
output_folder = r"C:\Users\Radcoflex-Purchase\Desktop\COLLEGE\SEM 7\Image and Video Analytics\lab 4\cola_segmented"


lower_green1 = (40, 100, 100)   
upper_green1 = (80, 255, 255)   

for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        
        frame = cv2.imread(os.path.join(input_folder, filename))
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower_green1, upper_green1)
        
        cv2.imwrite(os.path.join(output_folder, f"color_threshold_{filename}"), mask)

print("Segmentation completed and saved.")


Segmentation completed and saved.


Track objects

In [36]:
import cv2
import os
import numpy as np

input_folder = r"C:\Users\Radcoflex-Purchase\Desktop\COLLEGE\SEM 7\Image and Video Analytics\lab 4\cola_segmented"
output_folder = r"C:\Users\Radcoflex-Purchase\Desktop\COLLEGE\SEM 7\Image and Video Analytics\lab 4\cola_track"

prev_centroids = []

for filename in sorted(os.listdir(input_folder)):  
    if filename.endswith(".jpg") or filename.endswith(".png"):
        frame = cv2.imread(os.path.join(input_folder, filename))

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        _, mask = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        curr_centroids = []
        
        for contour in contours:
            if cv2.contourArea(contour) > 200:  
                x, y, w, h = cv2.boundingRect(contour)
                
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    curr_centroids.append((cX, cY))

                    cv2.circle(frame, (cX, cY), 5, (255, 0, 0), -1)

        if prev_centroids and curr_centroids:  
            for prev_centroid in prev_centroids:
                nearest_centroid = min(curr_centroids, key=lambda c: np.linalg.norm(np.array(c) - np.array(prev_centroid)))
                
                cv2.line(frame, prev_centroid, nearest_centroid, (0, 0, 255), 2)

        cv2.imwrite(os.path.join(output_folder, f"tracked_{filename}"), frame)

        prev_centroids = curr_centroids




In [38]:
import cv2
import os

input_folder = r"C:\Users\Radcoflex-Purchase\Desktop\COLLEGE\SEM 7\Image and Video Analytics\lab 4/cola_track"
output_folder = r"C:\Users\Radcoflex-Purchase\Desktop\COLLEGE\SEM 7\Image and Video Analytics\lab 4/cola_fore_back"

background_subtractor = cv2.createBackgroundSubtractorMOG2(history=50, varThreshold=70, detectShadows=True)

for filename in sorted(os.listdir(input_folder)): 
    if filename.endswith(".jpg") or filename.endswith(".png"):
        frame = cv2.imread(os.path.join(input_folder, filename))

        fg_mask = background_subtractor.apply(frame)

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
        fg_mask_3ch = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)

        background = cv2.bitwise_and(frame, frame, mask=cv2.bitwise_not(fg_mask))
        foreground = cv2.bitwise_and(frame, frame, mask=fg_mask)

        cv2.imwrite(os.path.join(output_folder, f"background_{filename}"), foreground)
        cv2.imwrite(os.path.join(output_folder, f"foreround_{filename}"), background)

print("Foreground-background segmentation completed and saved.")


Foreground-background segmentation completed and saved.


scene cut


In [15]:
import cv2
import os
import numpy as np

input_folder = r"C:\Users\Radcoflex-Purchase\Desktop\COLLEGE\SEM 7\Image and Video Analytics\lab 4/cola"


pixel_diff_threshold = 30  
hist_diff_threshold = 0.7 


def compare_histograms(frame1, frame2):
    hsv_frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2HSV)
    hsv_frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)
    
    hist_frame1 = cv2.calcHist([hsv_frame1], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
    hist_frame2 = cv2.calcHist([hsv_frame2], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
    
    cv2.normalize(hist_frame1, hist_frame1)
    cv2.normalize(hist_frame2, hist_frame2)
    
    return cv2.compareHist(hist_frame1, hist_frame2, cv2.HISTCMP_CORREL)

def compute_pixel_difference(frame1, frame2):
    diff = cv2.absdiff(frame1, frame2)
    mean_diff = np.mean(diff)
    return mean_diff

prev_frame = None
frame_number = 0
scene_cuts = []

for filename in sorted(os.listdir(input_folder)): 
    if filename.endswith(".jpg") or filename.endswith(".png"):
        frame = cv2.imread(os.path.join(input_folder, filename))

        if prev_frame is not None:
            pixel_diff = compute_pixel_difference(prev_frame, frame)
            
            hist_diff = compare_histograms(prev_frame, frame)
            
            if pixel_diff > pixel_diff_threshold or hist_diff < hist_diff_threshold:
                print(f"Scene cut detected at frame {frame_number} (Pixel diff: {pixel_diff}, Hist diff: {hist_diff})")
                scene_cuts.append(frame_number)
        
        prev_frame = frame
        frame_number += 1

print(f"Scene cuts detected at frames: {scene_cuts}")


Scene cut detected at frame 10 (Pixel diff: 93.35308195891204, Hist diff: 0.030984378976063365)
Scene cut detected at frame 43 (Pixel diff: 53.2844798900463, Hist diff: 0.8236799999363161)
Scene cut detected at frame 76 (Pixel diff: 57.00165183738426, Hist diff: 0.1757649597841487)
Scene cut detected at frame 78 (Pixel diff: 36.67390480324074, Hist diff: 0.965284872782859)
Scene cut detected at frame 79 (Pixel diff: 57.86677915219907, Hist diff: 0.9639672718046816)
Scene cut detected at frame 127 (Pixel diff: 49.65019133391203, Hist diff: 0.7543759063986122)
Scene cut detected at frame 147 (Pixel diff: 89.03747432002315, Hist diff: 0.5082651700293114)
Scene cut detected at frame 195 (Pixel diff: 39.39852394386574, Hist diff: 0.8788859145191895)
Scene cut detected at frame 196 (Pixel diff: 68.46531105324074, Hist diff: 0.7563525293735641)
Scene cut detected at frame 197 (Pixel diff: 63.73228479456019, Hist diff: 0.8575594195647827)
Scene cut detected at frame 198 (Pixel diff: 61.5078519

In [34]:
import cv2
import os
import numpy as np

input_folder = r"C:\Users\Radcoflex-Purchase\Desktop\COLLEGE\SEM 7\Image and Video Analytics\lab 4\cola"

intensity_change_threshold = 20  
histogram_similarity_threshold = 0.7 
window_size = 20 

def compute_average_intensity(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return np.mean(gray_frame)

def compare_histograms(frame1, frame2):
    hsv_frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2HSV)
    hsv_frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)

    hist_frame1 = cv2.calcHist([hsv_frame1], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
    hist_frame2 = cv2.calcHist([hsv_frame2], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
    
    cv2.normalize(hist_frame1, hist_frame1)
    cv2.normalize(hist_frame2, hist_frame2)
    
    return cv2.compareHist(hist_frame1, hist_frame2, cv2.HISTCMP_CORREL)

frame_number = 0
soft_cuts = []
intensities = []

for filename in sorted(os.listdir(input_folder)): 
    if filename.endswith(".jpg") or filename.endswith(".png"):
        frame = cv2.imread(os.path.join(input_folder, filename))

        if frame is not None:
            avg_intensity = compute_average_intensity(frame)
            intensities.append(avg_intensity)

            if len(intensities) > window_size:
                intensity_diffs = [abs(intensities[i] - intensities[i+1]) for i in range(len(intensities) - 1)]
                
                mean_intensity_diff = np.mean(intensity_diffs[-window_size:])
                median_intensity_diff = np.median(intensity_diffs[-window_size:])

                hist_similarity = compare_histograms(frame, cv2.imread(os.path.join(input_folder, sorted(os.listdir(input_folder))[frame_number - 1])))

                if mean_intensity_diff < intensity_change_threshold and hist_similarity < histogram_similarity_threshold:
                    soft_cuts.append(frame_number - window_size) 
                    print(f"Soft cut detected at frame {frame_number - window_size}")

            frame_number += 1

print(f"Soft cuts detected at frames: {soft_cuts}")


Soft cut detected at frame 56
Soft cut detected at frame 127
Soft cut detected at frame 179
Soft cut detected at frame 215
Soft cut detected at frame 217
Soft cut detected at frame 260
Soft cut detected at frame 263
Soft cut detected at frame 280
Soft cut detected at frame 281
Soft cuts detected at frames: [56, 127, 179, 215, 217, 260, 263, 280, 281]
